In [ ]:
import pandas as pd
import numpy as np
import requests
import os
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from datetime import datetime

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool
from langchain_core.messages import AIMessage, ToolMessage
from langchain.agents import create_agent

from IPython.display import Image, Markdown, display
import markdown
import webbrowser

In [ ]:
%pip install matplotlib
%pip install langchain langchain_openai
%pip install markdown

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## 모델 및 경로 설정

In [ ]:
model = ChatOpenAI(model = 'gpt-4o-mini',
                   temperature=0.3,
                   api_key='/')

In [ ]:
file_path = "C:/Users/thswj/Desktop/aa/"

## 필요 함수, Tool, 에이전트 정의

In [ ]:
# 분석 함수
def analysis_data():
    df = pd.read_csv('C:/Users/thswj/Desktop/aa/btc_data.csv')

    # 수익률
    df['return'] = round((df['close'] - df['open']) / df['open'] * 100, 2)

    # 변동률
    df['volatility'] = round((df['high'] - df['low']) / df['open'] * 100, 2)

    # 5분 이동 평균
    df['ma5'] = round(df['close'].rolling(window=5).mean(), 2)

        # RSI(상대 강도 지수)
    delta = df["close"].diff()
    gain = delta.clip(lower = 0)
    loss = -delta.clip(upper = 0)
    avg_gain = gain.rolling(window = 14, min_periods = 1).mean()
    avg_loss = loss.rolling(window = 14, min_periods = 1).mean()
    rs = avg_gain / (avg_loss.replace(0, np.nan))
    df["rsi"] = round(100 - (100 / (1 + rs)), 2)

    # 거래량 변화율(Volume %)
    df["volume_change"] = round(df["volume"].pct_change() * 100, 2)

    # 거래 금액
    df["volume_price"] = round(df["volume"] * df["close"], 2)

    # 평균 거래량
    df["avg_volume"] = round(df["volume"].mean(), 2)

    summary = {
        "mean_return" : round(df["return"].mean(), 2),
        "mean_volatility" : round(df["volatility"].mean(), 2),
        "Avg_RSI" : round(df["rsi"].iloc[-1], 2),
        "Avg_volume" : round(df["volume_price"].mean(), 2),
        "current_close" : round(df["close"].iloc[-1], 2),
        "timestamp" : datetime.now().strftime("%Y-%m-%d %H:%M")
    }

    return df, summary

# 출력용 함수
def analysis_data_summary(_input = None):
    _, summary = analysis_data()
    return summary

# 시각화 함수
def create_chart(_input = None):
    df, _ = analysis_data()
    plt.figure(figsize = (25, 15))
    plt.plot(df["timestamp"], df["close"], label = "Close Price", color = "blue")
    plt.plot(df["timestamp"], df["ma5"], label = "5_MA", color = "orange")
    plt.title("BTC Price & 5MA Trend")
    plt.xlabel("Time")
    plt.ylabel("Price (USD)")
    plt.legend()
    plt.xticks(rotation = 45)
    plt.tight_layout()

    filename = file_path + f"images/chart_{datetime.now().strftime('%H%M')}.png"
    plt.savefig(filename)
    plt.close()

    return filename

# 보고서 생성 함수(Tool용)
def generate_report(_input = None):
    _, summary = analysis_data()

    prompt = PromptTemplate.from_template(
        """최근 비트코인 시세 데이터를 요약한 통계값은 다음과 같습니다.
        - 평균 수익률 : {mean_return}%
        - 평균 변동률 : {mean_volatility}%
        - RSI : {Avg_RSI}
        - 평균 거래량 : {Avg_volume}
        - 현재 종가 : {current_close} USD
        - 분석 시각 : {timestamp}

        1. 최근 추세를 상승/하락/횡보 중 하나로 판단하고 근거를 제시해줘.
        2. 거래량과 변동성 간의 관계를 분석해줘.
        3. 단기 투자자 관점에서 리스크 요인을 2가지 제시해줘.
        4. 다음 시점(5분 후)에 대한 간략한 가격 전망을 작성해줘.
        5. 마지막으로 "요약 및 결론" 한 문장을 작성해줘.

        전문적이고 보고서 스타일의 톤으로, 섹션별로 Markdown 형식으로 구분해줘.
        """
    )

    report_prompt = prompt.format(**summary)
    report = model.invoke(report_prompt).content

    return report

In [ ]:
# Tool 정의
tools = [
    Tool(
        name = 'Analysis_Data',
        func = analysis_data_summary,
        description = '비트코인 데이터 통계분석'
    ),
    Tool(
        name = "Create_Chart",
        func = create_chart,
        description = "비트코인 가격 차트 이미지"
    ),
    Tool(
        name = "Generate_Report",
        func = generate_report,
        description = "AI 보고서 생성"
    )
]

In [ ]:
# 에이전트 구성
prompt = """당신은 비트코인 시장 분석 전문가입니다.
도구를 적절히 활용해 아래 요청에 답변하세요.
도구 사용이 끝나면 결과를 전문 보고서 형태로 Markdown으로 정리하세요."""

agent = create_agent(model=model, tools=tools, system_prompt=prompt)

## 메인 루프

In [ ]:
# 메인 루프
if __name__ == "__main__":
    # 데이터 분석 테스트
    df, summary = analysis_data()
    print("\n [데이터 통계 요약]")
    for k, v in summary.items():
        print(f"- {k}: {v}")

    # 시각화 테스트
    chart_path = create_chart()
    print(f"\n 차트 파일 생성 완료: {chart_path}")

    # Agent 테스트
    print("\n [LangChain Agent 작동 테스트 중...]")
    response = agent.invoke({"input" : "비트코인 시장 분석 보고서를 생성해줘."})
    messages = response['messages']

    # content 추출
    ai_messages = None
    for msg in reversed(messages):
        if isinstance(msg, AIMessage) and msg.content:
            ai_message = msg.content
            break
    # ai_messages = next((m for m in reversed(messages) if isinstance(m, AIMessage) and m.content), None)

    tool_message = None
    for msg in reversed(messages):
        if isinstance(msg, ToolMessage) and msg.content:
            tool_message = msg.content
            break
    # last_tool = next((m for m in reversed(messages) if isinstance(m, ToolMessage) and m.content), None)

    final_report = "" + ai_message + "\n\n---\n\n" + tool_message

    print("\n [AI 보고서 결과]\n")
    print(final_report)

    # 스크립트 종료
    print("\n 프로그램 종료")


 [데이터 통계 요약]
- mean_return: 0.02
- mean_volatility: 0.19
- Avg_RSI: 32.74
- Avg_volume: 7103544.17
- current_close: 92246.36
- timestamp: 2025-11-21 13:01

 차트 파일 생성 완료: C:/Users/thswj/Desktop/aa/images/chart_1301.png

 [LangChain Agent 작동 테스트 중...]

 [AI 보고서 결과]

# 비트코인 시장 분석 보고서

## 1. 개요
비트코인은 최근 시장에서 중요한 자산으로 자리잡고 있으며, 본 보고서는 비트코인의 최근 통계 데이터와 가격 차트를 분석하여 현재 시장 상황을 평가합니다.

## 2. 비트코인 통계 분석
- **평균 수익률**: 2%
- **평균 변동성**: 19%
- **평균 상대 강도 지수 (RSI)**: 32.74
- **평균 거래량**: 7,103,544.17 BTC
- **현재 종가**: 92,246.36 USD
- **분석 시점**: 2025년 11월 21일 13:01

## 3. 가격 차트
아래는 비트코인의 최근 가격 차트입니다.

![Bitcoin Price Chart](C:/Users/thswj/Desktop/aa/images/chart_1301.png)

## 4. 결론
비트코인의 평균 수익률은 2%로 긍정적인 신호를 보이고 있으나, 평균 변동성이 19%로 높은 편입니다. RSI 값이 32.74로 낮은 상태이므로 과매도 구간에 있을 가능성이 있습니다. 평균 거래량은 상당히 활발하여 시장의 유동성이 높음을 나타냅니다. 현재 종가는 92,246.36 USD로, 시장의 전반적인 추세를 반영하고 있습니다.

## 5. 최근 추세 판단
현재 비트코인의 평균 수익률이 0.02%로 낮고, RSI(상대강도지수)가 32.74로 과매도 구간에 가까운 상태입니다. 이러한 지표들은 비트코인이 상승세보다는 하락세에 가까운 상태임을 나타냅니다. RSI가 30 이하일 경우

## 저장

In [ ]:
def convert_path(final_report, mode):
    if mode == 'notebook':
        return final_report.replace(
            'C:/Users/thswj/Desktop/aa/images/chart_1301.png',
            'aa/images/chart_1301.png')
    elif mode == 'md/html':
        return final_report.replace(
            'C:/Users/thswj/Desktop/aa/images/chart_1301.png',
            'images/chart_1301.png')

In [ ]:
final_report_notebook = convert_path(final_report, 'notebook')
display(Markdown(final_report_notebook))

# 비트코인 시장 분석 보고서

## 1. 개요
비트코인은 최근 시장에서 중요한 자산으로 자리잡고 있으며, 본 보고서는 비트코인의 최근 통계 데이터와 가격 차트를 분석하여 현재 시장 상황을 평가합니다.

## 2. 비트코인 통계 분석
- **평균 수익률**: 2%
- **평균 변동성**: 19%
- **평균 상대 강도 지수 (RSI)**: 32.74
- **평균 거래량**: 7,103,544.17 BTC
- **현재 종가**: 92,246.36 USD
- **분석 시점**: 2025년 11월 21일 13:01

## 3. 가격 차트
아래는 비트코인의 최근 가격 차트입니다.

![Bitcoin Price Chart](aa/images/chart_1301.png)

## 4. 결론
비트코인의 평균 수익률은 2%로 긍정적인 신호를 보이고 있으나, 평균 변동성이 19%로 높은 편입니다. RSI 값이 32.74로 낮은 상태이므로 과매도 구간에 있을 가능성이 있습니다. 평균 거래량은 상당히 활발하여 시장의 유동성이 높음을 나타냅니다. 현재 종가는 92,246.36 USD로, 시장의 전반적인 추세를 반영하고 있습니다.

## 5. 최근 추세 판단
현재 비트코인의 평균 수익률이 0.02%로 낮고, RSI(상대강도지수)가 32.74로 과매도 구간에 가까운 상태입니다. 이러한 지표들은 비트코인이 상승세보다는 하락세에 가까운 상태임을 나타냅니다. RSI가 30 이하일 경우 과매도 상태로 해석되며, 이는 가격이 하락할 가능성이 높음을 시사합니다. 따라서, 최근 비트코인의 추세는 하락으로 판단됩니다.

## 6. 거래량과 변동성 간의 관계 분석
평균 거래량이 7,103,544.17로 나타나며, 평균 변동률이 0.19%로 상대적으로 낮은 편입니다. 일반적으로 높은 거래량은 시장의 관심과 참여를 나타내며, 이는 가격 변동성을 증가시킬 수 있습니다. 그러나 현재의 거래량과 변동률을 고려할 때, 거래량이 낮거나 보합세를 보이고 있는 상황에서 변동성이 낮은 것은 시장의 관심이 감소하고 있음을 나타냅니다. 이는 투자자들이 비트코인에 대한 확신이 부족하여 거래를 주저하고 있다는 신호일 수 있습니다.

## 7. 단기 투자자 관점에서의 리스크 요인
1. **시장 변동성**: 현재 비트코인의 평균 변동률이 0.19%로 낮지만, 시장의 불확실성이 커질 경우 급격한 가격 변동이 발생할 수 있습니다. 이는 단기 투자자에게 큰 손실을 초래할 수 있는 리스크입니다.
   
2. **거래량 감소**: 평균 거래량이 낮은 상태는 시장의 유동성이 떨어지고 있음을 의미합니다. 이는 가격의 급격한 하락이나 상승에 대한 저항이 약해질 수 있으며, 단기 투자자에게 불리한 상황을 초래할 수 있습니다.

## 8. 다음 시점(5분 후) 가격 전망
현재 비트코인의 종가가 92,246.36 USD이며, RSI가 과매도 구간에 가까운 상태입니다. 단기적으로 가격은 소폭 하락할 가능성이 높습니다. 그러나 과매도 상태로 인해 반등의 가능성도 존재하므로, 가격은 92,000 USD에서 92,300 USD 사이에서 움직일 것으로 예상됩니다.

## 9. 요약 및 결론
현재 비트코인은 하락세에 있으며, 낮은 거래량과 변동성으로 인해 단기 투자자에게 리스크가 존재하는 상황입니다.

---

# 비트코인 시세 분석 보고서

## 1. 최근 추세 판단
현재 비트코인의 평균 수익률이 0.02%로 낮고, RSI(상대강도지수)가 32.74로 과매도 구간에 가까운 상태입니다. 이러한 지표들은 비트코인이 상승세보다는 하락세에 가까운 상태임을 나타냅니다. RSI가 30 이하일 경우 과매도 상태로 해석되며, 이는 가격이 하락할 가능성이 높음을 시사합니다. 따라서, 최근 비트코인의 추세는 하락으로 판단됩니다.

## 2. 거래량과 변동성 간의 관계 분석
평균 거래량이 7,103,544.17로 나타나며, 평균 변동률이 0.19%로 상대적으로 낮은 편입니다. 일반적으로 높은 거래량은 시장의 관심과 참여를 나타내며, 이는 가격 변동성을 증가시킬 수 있습니다. 그러나 현재의 거래량과 변동률을 고려할 때, 거래량이 낮거나 보합세를 보이고 있는 상황에서 변동성이 낮은 것은 시장의 관심이 감소하고 있음을 나타냅니다. 이는 투자자들이 비트코인에 대한 확신이 부족하여 거래를 주저하고 있다는 신호일 수 있습니다.

## 3. 단기 투자자 관점에서의 리스크 요인
1. **시장 변동성**: 현재 비트코인의 평균 변동률이 0.19%로 낮지만, 시장의 불확실성이 커질 경우 급격한 가격 변동이 발생할 수 있습니다. 이는 단기 투자자에게 큰 손실을 초래할 수 있는 리스크입니다.
   
2. **거래량 감소**: 평균 거래량이 낮은 상태는 시장의 유동성이 떨어지고 있음을 의미합니다. 이는 가격의 급격한 하락이나 상승에 대한 저항이 약해질 수 있으며, 단기 투자자에게 불리한 상황을 초래할 수 있습니다.

## 4. 다음 시점(5분 후) 가격 전망
현재 비트코인의 종가가 92,246.36 USD이며, RSI가 과매도 구간에 가까운 상태입니다. 단기적으로 가격은 소폭 하락할 가능성이 높습니다. 그러나 과매도 상태로 인해 반등의 가능성도 존재하므로, 가격은 92,000 USD에서 92,300 USD 사이에서 움직일 것으로 예상됩니다.

## 5. 요약 및 결론
현재 비트코인은 하락세에 있으며, 낮은 거래량과 변동성으로 인해 단기 투자자에게 리스크가 존재하는 상황입니다.

In [ ]:
final_report_md_html = convert_path(final_report, 'md/html')

In [ ]:
final_report_md_html

'# 비트코인 시장 분석 보고서\n\n## 1. 개요\n비트코인은 최근 시장에서 중요한 자산으로 자리잡고 있으며, 본 보고서는 비트코인의 최근 통계 데이터와 가격 차트를 분석하여 현재 시장 상황을 평가합니다.\n\n## 2. 비트코인 통계 분석\n- **평균 수익률**: 2%\n- **평균 변동성**: 19%\n- **평균 상대 강도 지수 (RSI)**: 32.74\n- **평균 거래량**: 7,103,544.17 BTC\n- **현재 종가**: 92,246.36 USD\n- **분석 시점**: 2025년 11월 21일 13:01\n\n## 3. 가격 차트\n아래는 비트코인의 최근 가격 차트입니다.\n\n![Bitcoin Price Chart](images/chart_1301.png)\n\n## 4. 결론\n비트코인의 평균 수익률은 2%로 긍정적인 신호를 보이고 있으나, 평균 변동성이 19%로 높은 편입니다. RSI 값이 32.74로 낮은 상태이므로 과매도 구간에 있을 가능성이 있습니다. 평균 거래량은 상당히 활발하여 시장의 유동성이 높음을 나타냅니다. 현재 종가는 92,246.36 USD로, 시장의 전반적인 추세를 반영하고 있습니다.\n\n## 5. 최근 추세 판단\n현재 비트코인의 평균 수익률이 0.02%로 낮고, RSI(상대강도지수)가 32.74로 과매도 구간에 가까운 상태입니다. 이러한 지표들은 비트코인이 상승세보다는 하락세에 가까운 상태임을 나타냅니다. RSI가 30 이하일 경우 과매도 상태로 해석되며, 이는 가격이 하락할 가능성이 높음을 시사합니다. 따라서, 최근 비트코인의 추세는 하락으로 판단됩니다.\n\n## 6. 거래량과 변동성 간의 관계 분석\n평균 거래량이 7,103,544.17로 나타나며, 평균 변동률이 0.19%로 상대적으로 낮은 편입니다. 일반적으로 높은 거래량은 시장의 관심과 참여를 나타내며, 이는 가격 변동성을 증가시킬 수 있습니다. 그러나 현재의 거래량과 변동률을 고려할 때, 거래량이 낮거나 보합세를 보이고 있는 상황에서 변동성이

In [ ]:
# 마크다운 저장
def save_as_markdown(final_report_md_html, file_path= file_path + "btc_report.md"):
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(final_report_md_html)
    print(f"Markdown 파일 저장 완료: {file_path}")

save_as_markdown(final_report_md_html)

Markdown 파일 저장 완료: C:/Users/thswj/Desktop/aa/btc_report.md


In [ ]:
def save_as_html(final_report_md_html, html_path= file_path + "btc_report.html"):
    # Markdown > HTML
    html_content = markdown.markdown(final_report_md_html, extensions=["fenced_code", "tables"])

    # HTML 파일 저장
    with open(html_path, "w", encoding="utf-8") as f:
        f.write(html_content)

    print(f"HTML 파일 저장 완료: {html_path}")

    # 브라우저에서 열기
    # webbrowser.open("file://" + os.path.abspath(html_path))

save_as_html(final_report_md_html)

HTML 파일 저장 완료: C:/Users/thswj/Desktop/aa/btc_report.html


In [ ]:
# !jupyter nbconvert --to html --output "C:/Users/thswj/Desktop/새폴더/bts_report(nbconvert).html" "C:/Users/thswj/Desktop/데이터 자동화.ipynb"
# !jupyter nbconvert --to html --TemplateExporter.exclude_input=True "C:/Users/thswj/Desktop/데이터 자동화.ipynb"

In [ ]:
# !python client.py


서버 응답: 200 {"status":"success"}
